In [2]:
import pandas as pd
import xgboost as xgb

df = pd.read_csv("task3_recsys/alfabattle_data/preproc_data_for_boosting/preproc_data_for_boosting/merged_data.csv")

In [4]:
df.amnt_sum.sort_values()

167397       0.000000
814117       0.000000
662216       0.000000
409261       0.672689
15478        0.727422
             ...     
942690    2449.992391
36479     2615.102834
467834    2624.240617
43018     3694.431957
897004    4761.214120
Name: amnt_sum, Length: 963811, dtype: float64

In [5]:
pd.set_option("display.max_columns", None)

X = df.drop(columns=["app_id", "product", "flag"])
Y = df[["product"]]

In [6]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2, random_state=666, shuffle=True)

In [7]:
model = xgb.XGBClassifier(
    objective='multi:softmax',  
    num_class=len(Ytrain["product"].value_counts().index),                
    max_depth=6,                
    learning_rate=0.1,          
    n_estimators=256,           
    subsample=0.8,              
    colsample_bytree=0.8,       
    eval_metric='mlogloss'  
)
model.fit(Xtrain, Ytrain)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=256,
              n_jobs=None, num_class=5, num_parallel_tree=None, ...)

In [8]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score

Yhat_test = model.predict(Xtest)

accuracy = accuracy_score(Ytest, Yhat_test)
f1_macro, f1_weighted = f1_score(Ytest, Yhat_test, average="macro"), f1_score(Ytest, Yhat_test, average="weighted")
print(f"TEST:\nAccuracy: {accuracy:.2f}, F1-Macro: {f1_macro:.2f}, F1-Weighted: {f1_weighted:.2f}\nConfusion matrix:")

print(confusion_matrix(Ytest, Yhat_test), end="\nClf report:\n")
print(classification_report(Ytest, Yhat_test), end="\n")

# w/o cpt

TEST:
Accuracy: 0.55, F1-Macro: 0.32, F1-Weighted: 0.48
Confusion matrix:
[[90780  7135  2087   347   191]
 [41909  8450  1936   310    72]
 [ 7809  1474  6035    17    46]
 [10222  1627    74   924    12]
 [ 9739   670   373    81   443]]
Clf report:
              precision    recall  f1-score   support

           0       0.57      0.90      0.70    100540
           1       0.44      0.16      0.23     52677
           2       0.57      0.39      0.47     15381
           3       0.55      0.07      0.13     12859
           4       0.58      0.04      0.07     11306

    accuracy                           0.55    192763
   macro avg       0.54      0.31      0.32    192763
weighted avg       0.53      0.55      0.48    192763



In [26]:
pd.DataFrame([{k:v for k, v in zip(model.feature_names_in_, model.feature_importances_)}]).T.sort_values(0, ascending=False)

,0
days_before_max,0.051930
mean_operation_type_4,0.031132
mean_day_of_week_7,0.030622
mean_mcc_category_1,0.029646
mean_operation_type_2,0.025328
...,...
count_mcc_category_24,0.003191
count_operation_type_17,0.003144
mean_mcc_category_28,0.003086
count_operation_type_21,0.002866


In [ ]:
{}